In [3]:
import torch
import torchvision

import torch.distributed as dist


In [9]:
assert dist.is_available() and dist.is_nccl_available()
ngpus_per_node = torch.cuda.device_count()
print(ngpus_per_node)

4


In [5]:
dist.init_process_group(backend='nccl') # nccl for GPU

# mp.spawn(main_worker, nprocs=ngpus_per_node, args=(ngpus_per_node, ''))

ValueError: Error initializing torch.distributed using env:// rendezvous: environment variable RANK expected, but not set

In [5]:
efficientnet = torchvision.models.efficientnet_v2_s(weights='DEFAULT', progress=True, num_classes=1000) # 150MB, 20M params
# efficientnet = torchvision.models.efficientnet_v2_l(weights='DEFAULT', progress=True, num_classes=1000) # 455M, 118M params

In [3]:
print(efficientnet)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  

In [6]:
total_params = sum(p.numel() for p in efficientnet.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(p.numel() for p in efficientnet.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

21,458,488 total parameters.
21,458,488 training parameters.


In [3]:
from pytorch_toolbelt.utils import count_parameters
count_parameters(efficientnet)

{'total': 21458488, 'trainable': 21458488}

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [8]:
efficientnet = efficientnet.to(device)


In [9]:
path = '/data/yzhanglo/comp4471-project/data/crops/aabdnomlru/0_0.png'
pic = torchvision.io.read_image(path) # (C, H, W)
print(pic.shape, pic.dtype)

torch.Size([3, 466, 345]) torch.uint8


In [10]:
pic = torch.stack((pic, pic))
print(pic.shape, pic.dtype)

torch.Size([2, 3, 466, 345]) torch.uint8


In [11]:
pic = pic.to(torch.float)
print(pic.shape, pic.dtype)

torch.Size([2, 3, 466, 345]) torch.float32


In [12]:
pic = pic.to(device)

In [13]:
# pic.to(device).cuda()
with torch.no_grad():
    print(efficientnet(pic))

tensor([[-0.1765, -0.4661, -0.2673,  ...,  0.2628,  0.1716,  0.9043],
        [-0.0994,  0.2473,  0.1922,  ..., -0.3175, -0.0801,  0.4145]],
       device='cuda:0')


In [2]:
%cd /data/yzhanglo/comp4471-project

/data/yzhanglo/comp4471-project


In [3]:
from comp4471.main import main
main()

usage: Hello [-h] [--config CONFIG_FILE] [--workers WORKERS] [--fold FOLD]
             [--data-dir DATA_DIR] [--folds-csv FOLDS_CSV]
             [--crops-dir CROPS_DIR] [--output-dir OUTPUT_DIR]
             [--logdir LOGDIR]
Hello: error: unrecognized arguments: -f /homes/yzhanglo/.local/share/jupyter/runtime/kernel-30ae3234-7f65-45ec-8e07-5ce3b8d2e5df.json


SystemExit: 2

/data/yzhanglo/miniconda/envs/c4/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3386: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
